In [1]:
# Import Libraries

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [2]:
# Load the MNIST Dataset:

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(mnist, batch_size=64, shuffle=True)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 102787543.68it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 24407756.16it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 43778668.62it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2774214.18it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
# Define the Generator and Discriminator Models:

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input).view(-1, 1, 28, 28)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input.view(-1, 28*28))


In [4]:
# Initialize the Models and Optimizers:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

G = Generator().to(device)
D = Discriminator().to(device)

criterion = nn.BCELoss()
optimizerD = optim.Adam(D.parameters(), lr=0.0002)
optimizerG = optim.Adam(G.parameters(), lr=0.0002)


In [ ]:
# Training Loop:

num_epochs = 50
for epoch in range(num_epochs):
    for i, (real_images, _) in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)

        # Train Discriminator
        D.zero_grad()
        labels = torch.ones(batch_size, 1).to(device)
        outputs = D(real_images)
        d_loss_real = criterion(outputs, labels)
        d_loss_real.backward()

        noise = torch.randn(batch_size, 100).to(device)
        fake_images = G(noise)
        labels = torch.zeros(batch_size, 1).to(device)
        outputs = D(fake_images.detach())
        d_loss_fake = criterion(outputs, labels)
        d_loss_fake.backward()

        optimizerD.step()

        # Train Generator
        G.zero_grad()
        labels = torch.ones(batch_size, 1).to(device)
        outputs = D(fake_images)
        g_loss = criterion(outputs, labels)
        g_loss.backward()

        optimizerG.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss_real.item() + d_loss_fake.item()}, g_loss: {g_loss.item()}')

    if (epoch+1) % 10 == 0:
        with torch.no_grad():
            fake_images = G(noise).detach().cpu()
        plt.figure(figsize=(10,10))
        plt.imshow(fake_images[0].view(28,28), cmap='gray')
        plt.show()


Epoch [1/50], d_loss: 0.043877072632312775, g_loss: 10.02901554107666
Epoch [2/50], d_loss: 2.037524703890085, g_loss: 3.2585561275482178
Epoch [3/50], d_loss: 0.23366529494524002, g_loss: 3.8050451278686523
Epoch [4/50], d_loss: 0.06702527403831482, g_loss: 6.601181983947754
Epoch [5/50], d_loss: 0.12258262559771538, g_loss: 4.810296535491943
